In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import requests

In [18]:
response = requests.get('https://trendify-ui65.onrender.com/products')
data = response.json() 
df = pd.DataFrame(data)  #converting the JSON data into a pandas DataFrame.

In [19]:
df.head()

,wishingUsers,_id,price,details,category,productName,productImage
0,[],65aa9fea11b750a2db1ad9ab,"3,990.00","Sneakers with a combination of materials, piec...",shoes,DENIM SNEAKERS,https://static.zara.net/photos///2023/I/1/2/p/...
1,[65ab480079a0e18b8dc8b0bc],65aa9fea11b750a2db1ad9ac,2500.00,Monochrome trainers. Seven-eyelet facing. The ...,shoes,CHUNKY TRAINERS,https://static.zara.net/photos///2023/V/1/2/p/...
2,[],65aa9fea11b750a2db1ad9ad,"3,990.00",Sneakers. Plain upper. Seven-eyelet facing. Ch...,shoes,CHUNKY TRAINERS,https://static.zara.net/photos///2023/I/1/2/p/...
3,[],65aa9fea11b750a2db1ad9ae,"2,990.00","Trainers. Combination of materials, pieces and...",shoes,MULTIPIECE TRAINERS,https://static.zara.net/photos///2023/I/1/2/p/...
4,[],65aa9fea11b750a2db1ad9af,"7,990.00",Leather monk shoes. Two metal buckles. Leather...,shoes,LEATHER MONK SHOES,https://static.zara.net/photos///2023/I/1/2/p/...


In [20]:
df.isna().sum()

wishingUsers    0
_id             0
price           0
details         0
category        0
productName     0
productImage    0
dtype: int64

In [21]:
print(df.columns)


Index(['wishingUsers', '_id', 'price', 'details', 'category', 'productName',
       'productImage'],
      dtype='object')


In [23]:
df.loc[df.duplicated(subset='productName')] #duplicate rows in df based on the 'jobProfile' column

,wishingUsers,_id,price,details,category,productName,productImage
2,[],65aa9fea11b750a2db1ad9ad,"3,990.00",Sneakers. Plain upper. Seven-eyelet facing. Ch...,shoes,CHUNKY TRAINERS,https://static.zara.net/photos///2023/I/1/2/p/...
9,[],65aa9fea11b750a2db1ad9b4,"4,990.00",Monochrome ankle boots. Matching side gores an...,shoes,CHELSEA BOOTS,https://static.zara.net/photos///2023/I/1/2/p/...
29,[],65aabfdd11b750a2db1ad9c8,899.00,Loose-fitting T-shirt with a round neck and sh...,tShirt,SLOGAN PRINT T-SHIRT,https://static.zara.net/photos///2023/I/0/2/p/...
35,[],65aac38c11b750a2db1ad9cf,"7,590.00",Collared jacket with long sleeves and elastica...,jacket,FAUX LEATHER JACKET,https://static.zara.net/photos///2023/I/0/2/p/...
39,[],65aac38c11b750a2db1ad9d3,"4,990.00",Varsity jacket made of lightweight fabric. Jet...,jacket,BOMBER JACKET WITH PATCH,https://static.zara.net/photos///2023/I/0/2/p/...
53,[],65aac50211b750a2db1ad9e1,"2,490.00",Cropped slim fit jeans with a five-pocket desi...,pant,CROPPED SLIM JEANS,https://static.zara.net/photos///2023/I/0/1/p/...


In [24]:

df_copy1 =df.drop(columns=[ 'wishingUsers','_id']).copy()

In [ ]:
df_copy1

In [26]:
df_copy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   price         61 non-null     object
 1   details       61 non-null     object
 2   category      61 non-null     object
 3   productName   61 non-null     object
 4   productImage  61 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB


In [32]:

# Extract details and product names
details = [item['details'] for item in data]
product_names = [item['productName'] for item in data]

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(details)

# Compute cosine similarity
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

def recommend_products(user_input):
    # Find the index of the product in the data
    index = product_names.index(user_input)

    # Get the cosine similarity scores for the given product
    sim_scores = list(enumerate(cosine_similarities[index]))

    # Sort the products based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 3 similar products
    top_products = sim_scores[1:4]

    # Get the indices of the top products
    top_indices = [i[0] for i in top_products]

    # Return the recommended products
    return [(data[i]['productName'], data[i]['price'], data[i]['productImage'],data[i]['details']) for i in top_indices]

# Example usage
user_input = "DENIM SNEAKERS"
recommendations = recommend_products(user_input)

# Display recommendations
for recommendation in recommendations:
    print(f"Product Name: {recommendation[0]}\nPrice: {recommendation[1]}\ndetails: {recommendation[2]}\nProduct Image: {recommendation[3]}\n")


Product Name: MULTIPIECE TRAINERS
Price: 2,990.00
details: https://static.zara.net/photos///2023/I/1/2/p/2236/220/202/2/w/448/2236220202_2_1_1.jpg?ts=1692608383106
Product Image: Trainers. Combination of materials, pieces and colours on the upper. Six-eyelet facing. Irregular and chunky sole.

Product Name: CHUNKY TRAINERS
Price: 3,990.00
details: https://static.zara.net/photos///2023/I/1/2/p/2245/220/001/2/w/448/2245220001_6_1_1.jpg?ts=1692019633659
Product Image: Sneakers. Plain upper. Seven-eyelet facing. Chunky sole with irregular design at the back.

Product Name: CHUNKY TRAINERS
Price: 2500.00
details: https://static.zara.net/photos///2023/V/1/2/p/2206/120/802/2/w/448/2206120802_2_1_1.jpg?ts=1683033215969
Product Image: Monochrome trainers. Seven-eyelet facing. The laces are semi-hidden under two pieces, one on each side of the instep. Chunky sole with an irregular design.

